In [142]:
#import json
#import urllib
#import pandas as pd
#import numpy as np
#from pandas import Series, DataFrame

from pandas import DataFrame
import pandas as pd
import csv
from nltk import word_tokenize, wordpunct_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

In [201]:
twlist = []

In [202]:
with open(r"C:\\Users\\Student21\\Documents\\Training\\Weather Project\\trained_weather_2.csv", "r") as t:
    tweets_raw = pd.read_csv(t)

In [203]:
tweets = tweets_raw[['text', 'weather']].values.tolist()

In [204]:
tweets

[["RT @KenleyHargettjr: IT'S COLD! #Weather #CBS19 https://t.co/9j7DcDOeZv",
  'yes'],
 ['RT @Foolish4Harry: Somewhere on the polar ice cap, a tiny flower is emerging from the frozen tundra because Harry is the literal sun \xe2\x98\x80\xef\xb8\x8f htt\xe2\x80\xa6',
  'no'],
 ['RT @feltman41: APNewsBreak: Fishing groups seek Atlantic wind farm delay (from @AP) https://t.co/fClxe1yc90',
  'no'],
 ['@2_chairz this cold? https://t.co/19ElqwlUZd', 'no'],
 ['RT @lisajariffin: Finally had courage to speak to really hot gym guy. \xf0\x9f\x94\xa5 Turns out we have loads in common! For example, he likes men too.',
  'no'],
 ['#NoFilmsForSingleScreens girls aloud xxx pics hot naked men', 'no'],
 ["Let's go play in the snow at Winter Wonderfest @discoverycubeOC this holiday! #DCOCWinter via @Cleverlycathryn  https://t.co/1dxbQVI6G5",
  'no'],
 ['Rita Ora Performs at HOT 99.5s Jingle Ball 2015 in Washington, D.C. https://t.co/BphQncsfoc #RitaOra',
  'no'],
 ['RT @I_20ve980: \xe3\x82\xa2\xe3\x83\xb3

In [206]:
twlist = [tuple(l) for l in tweets] # turn nested list of lists into list of tuples
twtokens = []

In [207]:
for (words, weather) in twlist:
    words_filtered = [e.lower().decode('utf8') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    twtokens.append((words_filtered, weather))

In [208]:
twtokens

[([u"it's", u'cold!', u'#weather', u'#cbs19'], 'yes'),
 ([u'somewhere',
   u'the',
   u'polar',
   u'ice',
   u'cap,',
   u'tiny',
   u'flower',
   u'emerging',
   u'from',
   u'the',
   u'frozen',
   u'tundra',
   u'because',
   u'harry',
   u'the',
   u'literal',
   u'sun',
   u'\u2600\ufe0f',
   u'htt\u2026'],
  'no'),
 ([u'fishing',
   u'groups',
   u'seek',
   u'atlantic',
   u'wind',
   u'farm',
   u'delay',
   u'(from',
   u'@ap)'],
  'no'),
 ([u'@2_chairz', u'this', u'cold?'], 'no'),
 ([u'finally',
   u'had',
   u'courage',
   u'speak',
   u'really',
   u'hot',
   u'gym',
   u'guy.',
   u'\U0001f525',
   u'turns',
   u'out',
   u'have',
   u'loads',
   u'common!',
   u'for',
   u'example,',
   u'likes',
   u'men',
   u'too.'],
  'no'),
 ([u'girls', u'aloud', u'xxx', u'pics', u'hot', u'naked', u'men'], 'no'),
 ([u"let's",
   u'play',
   u'the',
   u'snow',
   u'winter',
   u'wonderfest',
   u'this',
   u'holiday!',
   u'via'],
  'no'),
 ([u'rita',
   u'ora',
   u'performs',
   u

In [209]:
import nltk
def get_words_in_tweets(tweets):
    all_words = []
    for (words, weather) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [210]:
word_features = get_word_features(get_words_in_tweets(twtokens))

In [211]:
word_features

[u'raining',
 u'@hellamur',
 u'1,6',
 u'(photos)',
 u'aerry',
 u'@gfanmily:',
 u'1,0',
 u'four',
 u'duendes',
 u"friend's",
 u'befikre',
 u'todays',
 u'@femaies:',
 u'conjure',
 u'increase',
 u'canes',
 u'shura',
 u'snowing',
 u'powdery',
 u'apimetrics',
 u'igual',
 u'shuru',
 u'correcto',
 u'ticket\u2026',
 u'lord',
 u'immature',
 u'(((every.',
 u'1,4',
 u'caramel.',
 u'mlb',
 u'better.',
 u'sehun',
 u'oceans',
 u'dell',
 u'vixx',
 u'advice,',
 u'(hot',
 u'yellow',
 u'advice:',
 u'\u062a\u0645\u0635\u0644\u0647',
 u'bringing',
 u'27\xb0',
 u'liaisons',
 u'prize',
 u'#avi',
 u'bcbg',
 u'wednesday',
 u'ooop',
 u'haben',
 u'boynundan',
 u'@spaikin',
 u'this....',
 u'basico',
 u'snuggley',
 u'(except',
 u'her.',
 u'her,',
 u'miller',
 u'hanging',
 u'bacon',
 u'peaks",',
 u'canci\xf3n',
 u'budget',
 u'tires',
 u'second',
 u'apid:',
 u'274',
 u'spoken;',
 u'denied.\u2026',
 u'intellect,',
 u'\u306e',
 u'quite',
 u'blouse',
 u'sesame',
 u'brexit',
 u'report:',
 u'thunder',
 u'kid:',
 u'cooki

In [212]:
# The Classifier
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features


In [213]:
training_set = nltk.classify.apply_features(extract_features, tweets)

In [214]:
training_set

[({u'contains(corporate)': False, u'contains(waste)': False, u'contains(am,)': False, u'contains(frightful)': False, u'contains(habang)': False, u'contains(-rain)': False, u'contains(huy\u1ec0n)': False, u'contains(rent)': False, u'contains(poblete)': False, u'contains(nature\u2026)': False, u'contains(stri\u2026)': False, u'contains(down!)': False, u'contains(gore\u2019s)': False, u'contains(smelling)': False, u'contains(stoneflies)': False, u'contains(lindesy)': False, u'contains(explains)': False, u'contains(sweaters)': False, u'contains(percentage)': False, u'contains(#beautiful)': False, u'contains(@wuthie16:)': False, u'contains(baro)': False, u'contains(joy!\U0001f382)': False, u'contains(raising)': False, u'contains(projects)': False, u'contains(airplane)': False, u'contains(thur-sun)': False, u'contains(ready)': False, u'contains(2016)': False, u'contains(!!!!!!!)': False, u'contains(poco)': False, u'contains(daly)': False, u'contains(fisura)': False, u'contains(9612)': False,

In [215]:
# train the classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [216]:
print classifier.show_most_informative_features(32)

Most Informative Features
         contains(elite) = False              no : yes    =      1.0 : 1.0
           contains(66%) = False              no : yes    =      1.0 : 1.0
        contains(tropez) = False              no : yes    =      1.0 : 1.0
      contains(forecast) = False              no : yes    =      1.0 : 1.0
           contains(art) = False              no : yes    =      1.0 : 1.0
     contains(@thedoors) = False              no : yes    =      1.0 : 1.0
          contains(🙏🏾) = False              no : yes    =      1.0 : 1.0
     contains(retweeter) = False              no : yes    =      1.0 : 1.0
        contains(#ctxwx) = False              no : yes    =      1.0 : 1.0
    contains(infinit...) = False              no : yes    =      1.0 : 1.0
       contains(finally) = False              no : yes    =      1.0 : 1.0
       contains(islam's) = False              no : yes    =      1.0 : 1.0
            contains(🍆) = False              no : yes    =      1.0 : 1.0
  

In [217]:
tweet1 = 'Man its warm. I wish we could get a little breeze or something'
print classifier.classify(extract_features(tweet1.split()))

yes


In [263]:
tweet2 = 'RT @priyankachopra: Thank is #JustSaying https://t.co/94M3bfM0Ha'
print classifier.classify(extract_features(tweet2.split()))

no


In [272]:
tweet3 = 'Hot chocolate with marshmallows please'
print classifier.classify(extract_features(tweet3.split()))

yes


In [277]:
tweet3 = '#JustSaying'
print classifier.classify(extract_features(tweet3.split()))

no
